# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [9]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
    print("網頁更新中...")
    # Scroll down to the bottom.
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    # Wait to load the page.
    time.sleep(2)

    # Calculate new scroll height and compare with last scroll height.
    new_height = browser.execute_script("return document.body.scrollHeight")
 
    if new_height == last_height:
        print("已達網頁最底端...")
        break
    last_height = new_height

網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
已達網頁最底端...


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [10]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [11]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 明天再加油！中油大降汽柴油價1元　98無鉛汽油每公升28.4元

▲ 中油下周一調降汽柴油價格。（圖／記者張一中攝）
記者李瑞瑾／台北報導
台灣中油公司宣布自明（10）日凌晨零時起調降汽、柴油價格每公升1元，調整後的參考零售價格為95無鉛汽油每公升26.4元、98無鉛汽油每公升28.4元，想加油的民眾不妨再等等，替自己荷包省點錢！
中油表示，受中國武漢地區肺炎疫情擴大，市場擔心恐將削弱全球需求成長因素影響，本週國際原油價格續跌。本週調價指標7D3B週均價為每桶54.32美元，較前週58.45美元下跌4.13美元，新臺幣兌美元匯率升值0.084元，國內油價依公式計算跌幅為5.86%，92無鉛汽油及柴油均未超過亞鄰最低價，故汽、柴油應各調降1元，足額調整。
調整後的各式油品參考零售價格，分別為92無鉛汽油每公升24.9元、95無鉛汽油每公升26.4元、98無鉛汽油每公升28.4元、超級柴油每公升22.7元，實際零售價格以各營業點公告為準。
----------------------------------------------------------------------

[1] 防疫股先緩緩　台股還有哪些題材可留意

▲ 台股。（示意圖／記者湯興漢攝）
記者李瑞瑾／台北報導
武漢疫情衝擊投資人信心，也讓防疫概念股成為盤面上最亮眼的新星，但也已經漲了不少，台股分析師指出，投資人想搶短可以趁壓回時進場。除了防疫之外，鋼鐵股、電子股中的半導體IC封測，以及營建股等，在疫情影響中相對抗跌。
----------------------------------------------------------------------

[2] 影／iPhone 9售價399美元再獲背書！　強攻平價策略吸引i6用戶升級

▲ iPhone 9。（圖／翻攝自PGR）
記者李瑞瑾／綜合報導
距蘋果公司傳聞已久的iPhone 9（iPhone SE2）正式發布日可能只剩一個月左右的時間。在這一消息發布之前，市場上對新機的推測仍持續，Fast Company在本周的一份新報告中，證實了iPhone 9的起價為399美元。
可靠的蘋果

▲麗星郵輪「寶瓶星號」停靠基隆港。（圖／記者林振民攝）
記者楊絡懸／基隆報導
載有1738名旅客的麗星郵輪「寶瓶星號」8日停靠基隆港，受武漢肺炎疫情影響，中央流行疫情指揮中心已進行檢疫。兆豐保險表示，船上共有36位旅客投保「郵輪旅遊綜合保險」，符合旅程延誤條件，已經聯繫保戶，並啟動「旅程延誤補償」理賠事宜與快速理賠機制。
兆豐保險表示，麗星郵輪「寶瓶星號」原定2月7日下午4點返台，因武漢肺炎疫情延燒，延至8日上午始停靠基隆港。根據資料統計，船上共有36位旅客投保「郵輪旅遊綜合保險」，符合該保險商品的旅程延誤條件，因此將啟動旅程延誤補償；基於關懷客戶善盡企業社會責任的立場，兆豐保險主動清點出保戶後，也將主動聯繫保戶，並啟動快速理賠機制。
另因考量國際郵輪船上空間密閉、人員互動頻繁，屬高風險場域，中央流行疫情指揮中心也宣布，自2月6日禁止國際郵輪靠泊我國港口，期間為2月6日至3月31日。
全台灣獨家銷售「郵輪旅遊綜合保險」，兆豐保險表示，針對先前已投保「郵輪旅遊綜合保險」的保戶，此段期間，保戶因無法自台灣港口搭乘郵輪，保險公司將主動聯繫保戶，辦理保單退保及退費手續。
【延伸閱讀】
----------------------------------------------------------------------

[11] 2/16熄燈前的榮耀！和平青鳥書店獲「德國iF設計大獎」　跨界創新合作成得獎原因

記者林育綾／台北報導
充滿話題的「和平青鳥書店」，榮獲被稱為「世界3大設計獎」的2020年德國iF設計大獎（iF Design Award）！得獎原因包括「獨立書店跟建商的合作，打破常規裡建案接待中心的定型觀念，而是跨界融合，結合藝術展覽、演講、文化活動，為公眾帶來創新又愉悅的體驗」。而原本預計營業180天的書店，後來又延長近9個月，將於2月16日熄燈，青鳥書店創辦人蔡瑞珊接獲消息相當開心，透露「如果沒有延長營業，可能得不到這個獎。」
▲「達永秋鄉X和平青鳥」獲得2020年德國iF設計大獎。（圖／記者湯興漢攝）
青鳥書店與達永建設跨界合作，去年5月在台北市麟光站附近的寧靜社區，以可拆卸式的建材，重金打造「期間限定」的和平青鳥書店，樣品屋（建案接待中心）結合書店、咖啡餐飲的複合經營模式，成為全台首例的話題焦點，加上「消失的書店」名號吸引不少人造訪；而達永所推出預

▲ 台股。（示意圖／記者湯興漢攝）
記者李瑞瑾／台北報導
武漢疫情的憂慮讓股市陷入大幅震盪，台股本周波動劇烈，漲跌點都在百點上下，面對這樣的走勢，不僅投資人心臟要強，也更期待何時可以趨穩，分析師指出，台股短線上要持穩，並回歸基本面觀察並不容易，在一、兩個月之間，會很常出現百點震盪。
分析師指出，之前預測的台股震盪區間多是在10,000~11,000點，現在依位階調高一點，約在11,000~12,000震盪。分析師建議，投資人短線可以設個停損點，在11,000左右停損或是個股停損點。
----------------------------------------------------------------------

[21] 趁著口罩之亂　Whoscall：詐騙電話假冒健保局鎖卡騙個資

記者余弦妙／台北報導
武漢肺炎疫情延燒，政府為讓人人都有口罩可以用，實施口罩實名制，民眾需持健保卡到健保指定藥局購買口罩，詐騙集團看中民眾恐慌心態，趁機大行詐騙，根據Whoscall調查，最常見的詐騙手法，即是假冒健保局之名，行詐騙之實。
Whoscall表示，最近口罩之亂中，最常見的詐騙手法，就是假冒健保局之名，打電話給民眾謊稱：「健保卡被查出有不當使用情形，將被鎖卡，有疑問請撥9」。其次則是在社群網站上，聲稱可免費領取口罩，騙取民眾個資。短短幾天已有許多民眾反應接到相關詐騙來電。
Whoscall更指出，詐騙集團會不停變造號碼，甚或假冒正常號碼，藉此來降低民眾戒心，民眾也可以透過Whoscall的Android版本回報機制，在接到可疑來電時，主動回報，全民一起揪出詐騙不當行徑。
▼詐騙集團利用口罩之亂進行詐騙，點圖可放大。（圖／取自Whoscall臉書）
----------------------------------------------------------------------

[22] 找工作兼顧防疫！　勞動部提供120萬個線上職缺

▲勞動部。（圖／記者余弦妙攝）
記者余弦妙／台北報導
隨著年後轉職潮的來臨，2020開春將進入第一波求職旺季，考量新型冠狀病毒肺炎疫情，勞動部鼓勵有求職、求才需求的民眾，可多加利用「台灣就業通」網站及0800-777-888免付費服務專線，不出門也能獲得多元服務。
勞動部表示，「台灣就業通」網站集結了就業服務、職業訓練

圖文／鏡週刊


這次鴻海集團攜手裕隆集團合資155億元成立新公司，結合兩大集團實力，揮軍智慧汽車產業鏈，是否能替台灣沉悶已久的產業帶來新曙光呢？其實，鴻海集團早在十多年前就揮軍汽車電子產業，去年裕隆宣布組織改造時，就曾傳出劉揚偉親自到裕隆總部拜訪嚴陳莉蓮，雙方經過半年多的磋商，終於讓這樁美事成真。
「富士康3.0的目標，是要在2025年把鴻海的毛利率提升到10％，其中重要的一步棋就是投資包括電動車、數位健康、機器人三大未來事業。」鴻海董事長劉揚偉在去年法說會對外宣示。
今年鴻海尾牙前，鴻海公告將與歐系全球第四大汽車品牌飛雅特克萊斯勒（FCA）建立合資子公司（各自持股五成），進軍電動汽車市場震驚市場。時隔半個月之後，又宣布與台灣汽車業龍頭裕隆，合資新公司揮軍智慧汽車產業。
在鴻海尾牙後，劉揚偉也親自對媒體解析鴻海的佈局，不過他也重申先前創辦人郭台銘提到「鴻海不會投入整車製造」的說法，「以現在汽車市場來看，有充沛的產能，這情況不會去做整車組裝。」
劉揚偉表示，「在三大未來產業之中，FCA是第一個的計畫 ，未來（與FCA）最主要會在設計平台、零組件以及供應鏈管理上合作。」劉揚偉認為供應鏈管理一直是ICT（資訊與通信科技）產業的強項，「ICT各種方法都嘗試過，也練就又快又好又便宜的能力，這能力如果能夠帶到傳統產業，一定對汽車業有非常大的衝擊。」鴻海過去就曾入股中國鋰電池製造商寧德時代，在香港上市的連接器子公司鴻騰精密，也佈局電動汽車供應鏈。
這次鴻海與裕隆結親，其實早有跡可循。早在去年8月底，裕隆集團宣布擴大代工對象，開放多元接單後，又針對旗下負責研發的華創車電下達全數遷往苗栗三義廠區，效法嚴凱泰當年「廠辦合一」的做法，加速與工廠整合，徹底整頓汽車事業時，當時就有傳言，鴻海曾與裕隆密切接觸。「就在裕隆宣布組織改革後，劉董（劉揚偉）就曾出現在新店裕隆總部，就是上門向嚴董（嚴陳莉蓮）爭取合作機會。」知情人士當時對本刊透露。
據了解，當時裕隆集團積極要處理華創車電的去留，鴻海確實曾經考慮一次買下整個汽車研發與製造團隊，但收購金額雙方遲遲談不攏，隨後即傳出雙方談判觸礁，裕隆也陸續爭取其他品牌新代工業務。「但雙方仍持續接觸，並未斷了合作的念頭。」知情人士透露。
隨著今年初，鴻海與飛雅特克萊斯勒（FCA）宣布合資各自持股五成合資子公司之後，也帶動與裕隆集團間的談判進度，雙方終於在7日

▲Android R（又名Android 11）可能即將問世，目前傳出已經在Pixel 2 手機嘗試運行。（圖／取自Google）
記者謝仁傑／綜合報導
目前關於Android R的細節還很少，報導指出，Android R將會改進深色模式，讓深色模式可以套用在更多的應用程式上；同時也允許使用者設定開啟或關閉深色模式的時間。
此外，Google可能會改進Android R的藍牙，支援更大容量的檔案傳輸，即便開啟飛航模式，藍牙也不會被關閉；如此一來，Pixel Buds等無線藍牙耳機仍可與Android R機型保持連接不中斷。
按照慣例，谷歌每年五月在舊金山舉辦的Google I/O年度網路開發者大會，都會發佈新的作業系統，去年發表了Android Q 作業系統，去年9 月以Android 10 命名正式推出，也是呼應10 週年。
外界預期，今年將發表的Android R，最後可望以Android 11正式命名推出。
----------------------------------------------------------------------

[45] 中華信評：武漢肺炎疫情慘業　銀行放款量不到5%

▲中華信評。（圖／記者吳靜君攝）
記者吳靜君／台北報導
中華信評今（7）日指出，台灣金融業應該暫時可以抵禦武漢肺炎疫情所帶來的威脅，雖然台灣與中國大陸有密切的往來關係，但是新型冠狀病毒爆發到目前為止，對台灣金融業信用品質造成立即威脅性仍然有限。
交通、觀光業、住宿及餐飲業　銀行放款量不到5%
目前武漢肺炎疫情已經對中國大陸的工業產值、境內的貨物運輸與出口造成不利的影響，許多大型台商企業在此過程中也受到波及。中華信評分析師藍于涵表示，疫情對中國大陸造成的經濟衝擊顯而易見，但是台灣的銀行業有良好的資本緩衝可以協助抵禦此次爆發疫情所帶來的潛在影響。
中華信評指出，去年9月底台灣的銀行業第一類資本比率為11.84%，這次爆發疫情之前，台灣對中國大陸的曝險已經降低到期總資產4%左右，主要原因是銀行受到中美貿易戰升溫影響而逐步風險趨避。
股市波動　台灣壽險業受到的潛在衝擊較大
不過，報告也指出，隨著冠狀病毒疫情的發展台灣壽險業的信用結構對資本市場變動比較敏感，中華信評認為，台灣壽險業因為此次疫情爆發而感受到的潛在衝擊，會比其在罹病率方面的感受影響更大。
且台灣壽險業者

▲1月出口情形較原本預期差（圖／記者林世文攝）
記者吳靜君／台北報導
財政部今（7）日公布海關進出口情形，因為農曆春節工作天數減少關係，1月出口金額為250.7億美元，較去年同期減少7.6%。
但根據財政部去年預測，今年1月的出口金額年減在4.5%至6.5%，顯見較預期低。進口方面，1月的進口金額為216.1億美元，較去年同期減17.7%。出超金額為34.6億美元。
----------------------------------------------------------------------

[58] 口罩出口禁令恐再延　經濟部：日產能拚千萬片三月上線

▲隨著疫情持續擴大，預計23日到期的口罩出口禁令，外界預期將再度延長。（圖／記者湯興漢攝）
記者林淑慧／台北報導
口罩實名制上路，政府斥資2億元打造口罩新產線，預計三月起國內口罩日產能將達千萬片，隨著疫情持續擴大，預計23日到期的口罩出口禁令，外界預期將再度延長，對此經濟部官員回應表示，有待中央疫情指揮中心確認後宣佈。
武漢肺炎疫情持續延燒，近期口罩一罩難求，經濟部長沈榮津表示，國內口罩未來每週日產能可增加70萬片，兩週後可望達到460萬片。隨著60台新機具投入生產後，3月上旬產能可達到1000萬片。
全球瘋搶口罩，造成全球口罩荒，經濟部次長王美花進一步說明，因應這波防疫需求，讓全球看見台灣製造業的能量，在疫情爆發前，台灣名列全球口罩產能第三大國，在政府斥資2億元投資新產線後，預計3月起日產能將達千萬片，屆時台灣可望躍居全球口罩產能亞軍。
經濟部強調，目前每天三班戰情會議，隨時確認既有產線的產能，在廠商重新啟用封存機器，國軍也陸續補充人力後，目前每天可生產320萬片、下週將增加到390萬片/日、下下週再增加到460萬片/日，而製作口罩的原物料都已確認，國內供應無虞。
----------------------------------------------------------------------

[59] 台肥換董座！　由農委會副主委陳添壽2月12日接棒

▲農委會副主委陳添壽。（圖／記者許展溢攝）
記者余弦妙／台北報導
台肥（1722）今（7）日重大訊息公告「法人董事代表人異動」，原法人代表為董座康信鴻，現在則由行政院農委會副主委陳添壽12日接任。
據了解，去年3月市場就傳出台肥董座恐

▲武漢肺炎疫情蔓延尚未見遏止之勢，全球各地陸續傳出確診病例。（圖／達志影像／美聯社）
記者紀佳妘／台北報導
武漢肺炎疫情持續擴散，在大陸設有分行、子行的國銀今（7）日表示，配合當地政府相關規定開工，且各據點也啟動緊急應變措施，包括分組上班、追蹤員工身體健康情形等，也落實各項防疫措施。
富邦金控已公布「富邦金融控股股份有限公司暨子公司因應嚴重特殊傳染性肺炎之緊急應變機制」，提供全體同仁遵循，並全面啟動相關防疫機制，目前針對辦公及營業場所採取3大防疫措施。
首先，公司各棟辦公大樓全面對所有進出同仁、訪客進行量體溫及使用酒精消毒；第二，各子公司營業處第一線同仁，全面配戴口罩；第三，強化宣導請同仁以肥皂勤洗手、戴口罩，全面防疫，並落實同仁一旦有確診病例或不適症狀立即確實通報。
富邦金100%持股的子公司富邦華一銀行在大陸設有26個營業據點，到1月底為止，富邦華一台籍員工一共有151人。富邦華一銀行表示，全力配合做好疫情防控工作，開工時間依照各地政府相關規定執行，目前大多數分行恢復營業時間預計為2月10日。
富邦華一銀行指出，各分行也將遵循上述辦公及營業場所防疫措施，並且每日對辦公及營業場所、客戶接待區、辦公機具等進行消毒，同時配置防疫相關的消毒液、洗手液。另在疫情防控期間，富邦華一銀行採用線上軟體開會辦公，並暫緩員工各類出差。
在大陸擁有上海、廣州、廈門、深圳共5家分支機構的中國信託銀行表示，大陸各分行是依當地銀保監局、政府及勞動部門規定，通知員工分別於2月3日及2月10日開工。如當地監管單位另有防疫規定，或因往來交通受阻等特殊考量，得採自主隔離、在家工作。在自主隔離期間，公司正常給薪。
中國信託銀行也提及，目前大陸各分行採行分組上班、彈性工時，到班員工提供醫療口罩、乾洗酒精、每日量測體溫等防衛措施，並強化行舍辦公營業場所衛生管理，每日進行線上問卷調查以即時掌握員工疫情情況，管控可能的疫情風險。
----------------------------------------------------------------------

[73] 台積電去年專利申請首度突破千件　名列國內百大冠軍

▲經濟部智財局今（7）日公告2019年專利申請及公告發證統計排名，台積電去年的專利申請量以1333件名列本國企業冠軍。（圖／路透社）
記者林淑慧／台北報導
全台專利申請排名大公開

▲ 新台幣兌美元今天開盤貶值6.8分。（圖／路透社）
記者李瑞瑾／台北報導
台北外匯市場新台幣兌美元匯率今（7）日開盤貶值6.8分、以30.15元兌1美元開出。
新台幣兌美元昨（6）日盤中一度再現29字頭，但仍收在30.082兌1美元；今天開盤則貶值6.8分，以30.15元兌1美元開出。
新台幣開盤後收斂跌勢，截至上午9點09分，新台幣來到30.1元兌1美元，貶值1.8分。
----------------------------------------------------------------------

[85] 台股下跌逾百點！不受美股創高激勵　台股早盤跌破11700點

▲ 台股。（圖／記者湯興漢攝）
記者李瑞瑾／台北報導
中國大陸對美方釋出降稅利多，加上美國財報與經濟數據優於預期，美股三大指數齊刷歷史新高紀錄。台股卻不受美股激勵，在昨日大漲逾百點後拉回整理，今（7）日開盤下跌36點、以11712點開出。
截至上午9點04分，台股跌勢擴大，加權指數來到11,641點、下跌108點。
電子三雄中，台積電（2330）以330元開出、下跌0.75%；鴻海（2317）開在平盤83.6元；股王大立光（3008）以4780元開出、下跌1.34%。
中國大陸宣布調降對美商品關稅、美國強勁財報與經濟數據激勵，加上武漢肺炎對金融市場的影響漸小，美股道瓊、那指、標普同步刷新歷史新高紀錄。
美股道瓊指數上漲88.92點，或0.3%，收29,379.77點；那斯達克指數上漲63.47點，或0.67%，收9,572.15點；標普 500 指數上漲10.81點，或0.32%，收3,345.5點；費城半導體指數上漲0.12點，或0.01%，收1909.64點。
----------------------------------------------------------------------

[86] 賀！捷運環狀線通車　新埔民生站香榭京宴全新落成

圖、文／富比士提供
尤其是離站區僅三分鐘步行距離的「香榭京宴」，明顯地成為捷運通車最大得利者。除了既有的「新埔站」貫穿北市心臟的藍線大動脈，現在再加上360°包抄雙北市的環線系統「新埔民生站」，城市生活最重要的效率與自由就一手在握了，這還沒算上緊鄰的江翠國中與便利的江翠生活圈。
至於「香榭京宴」看不到的內涵，那就更厲害了；

▲根據統計，2019年國銀對中小企業放款餘額達4,599億元，創下史上次高水準。（示意圖／pixabay）
記者紀佳妘／台北報導
金管會今（6）日公布本國銀行對中小企業放款餘額，2019年放款餘額增加4,599億元，創下史上次高，目標達成率達170.34%，其中土銀、華南銀、合庫銀、台企銀、國泰世華銀等5家放款最多。
根據金管會統計，2019年本國銀行對中小企業放款餘額達6兆8,979億元，較2018年底大幅增加4,599億，一舉改寫歷史次高水準，僅次於2010年的4,721億元；中小企業放款戶數達29萬1,551戶，較2018年底的24萬8,473戶，年增4萬3,078戶。
金管會每年都會訂下國銀對中小企業放款目標，而2019年設定在新增2,700億元，累計目標達成率達170.34%，也就是超越當初設定的目標達成率。
金管會銀行局表示，2019年中小企業放款創下史上次高，主要是金管會新增放款戶數、小型和新創公司的考核目標與獎項，鼓勵銀行對中小企業放款。
觀察放款前5大銀行，2019年底相較於2018年底，對中小企業放款最多的銀行有3家，依序為土地銀行（606.83億元）、華南銀行（448.59億元）、合庫銀（437.36億元）、台企銀（348.93億元）及國泰世華銀（313.12億元）。
統計指出，2019年12月底中小企業放款餘額占全體企業放款餘額比率為62.87%，占民營企業放款餘額比率為65.61%，均較上月底的比率增加1.13個百分點。
另外，去年12月底中小企業放款的平均逾期放款比率為0.34%，較上月底減少0.02個百分點。金管會銀行局表示，將持續鼓勵銀行在兼顧風險控管下，對中小企業提供必要的營運資金，以營造有利中小企業融資環境。
